In [2]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

In [3]:
symbols = ['AAPL','MSFT','META','NVDA']
end_date = datetime.now()
start_date = datetime(2020,1,1)
data = vbt.YFData.download(
    symbols = symbols,    
    interval = '1d',
    start = start_date, 
    end = end_date,
).get('Close')
data

symbol,AAPL,MSFT,META,NVDA
Date,,,,
2019-12-31 05:00:00+00:00,70.920387,150.013397,203.992432,5.856667
2020-01-02 05:00:00+00:00,72.538513,152.791122,208.494675,5.971410
2020-01-03 05:00:00+00:00,71.833305,150.888611,207.391495,5.875831
2020-01-06 05:00:00+00:00,72.405670,151.278625,211.297409,5.900473
2020-01-07 05:00:00+00:00,72.065163,149.899307,211.754578,5.971908
...,...,...,...,...
2025-10-07 04:00:00+00:00,256.480011,523.979980,713.080017,185.039993
2025-10-08 04:00:00+00:00,258.059998,524.849976,717.840027,189.110001
2025-10-09 04:00:00+00:00,254.039993,522.400024,733.510010,192.570007


In [4]:
def _sma(data:pd.DataFrame, window: int):
    sma = data.rolling(window = window).mean()
    return sma

def apply_sma(data:pd.DataFrame, fast:int, slow:int):
    # print(data)
    fast_sma = _sma(data,window = fast)
    slow_sma = _sma(data,window = slow)
    trend = pd.DataFrame(0, index=data.index, columns=data.columns)
    trend[fast_sma < slow_sma] = -1
    trend [fast_sma > slow_sma] = 1 
    trend = trend.diff() #2 for buy, -2 for sell
    
    return fast_sma, slow_sma, trend

sma_indicator = vbt.IndicatorFactory(
    class_name = 'CustomSMA',
    short_name = 'SMA',
    input_names = ['data'],
    param_names = ['fast','slow'],
    output_names = ['fast_sma','slow_sma','trend' ]
).from_apply_func(apply_sma)

fast_windows = [10, 20, 30]
slow_windows = [50, 100, 150]

results = sma_indicator.run(data, 
                            fast = fast_windows, 
                            slow = slow_windows, 
                            keep_pd = True)
entries = results.trend == 2
exits = results.trend == -2
pf = vbt.Portfolio.from_signals(data, 
                                entries, 
                                exits,
                                fees=0.001,  
                                size=1_000,  # trade size per asset
                                init_cash=100_000
                                )
returns = pf.total_return()
print(returns)
print(returns.max())
print(returns.idxmax())





SMA_fast  SMA_slow  symbol
10        50        AAPL      0.983985
                    MSFT      0.485179
                    META      3.141509
                    NVDA      1.017864
20        100       AAPL      0.274060
                    MSFT      0.296764
                    META      4.371079
                    NVDA      1.524299
30        150       AAPL      0.107711
                    MSFT      0.769952
                    META      2.007826
                    NVDA      1.513946
Name: total_return, dtype: float64
4.37107873859698
(np.int64(20), np.int64(100), 'META')


In [ ]:
#Heatmap
fig =returns.vbt.heatmap(
    x_level = "SMA_slow",
    y_level = "SMA_fast",
    title='Total Return Heatmap: SMA',
    width=600,
    height=500
)
fig.show()


/home/erick-ngumo/Documents/vectorbt/.venv/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

